In [1]:
import os
import warnings
warnings.simplefilter(action = 'ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
def ignore_warn(*args, **kwargs):
    pass

warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

import pandas as pd
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.cm as cm

%matplotlib inline

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.5)
color = sns.color_palette()
sns.set_style('darkgrid')

from mpl_toolkits.mplot3d import Axes3D

import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode()

from scipy import stats
from scipy.stats import skew, norm, probplot, boxcox
from sklearn import preprocessing
import math

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import Orange
from Orange.data import Domain, DiscreteVariable, ContinuousVariable
from orangecontrib.associate.fpgrowth import *

In [2]:
# File location and type
file_location = "/FileStore/tables/data.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [3]:
cs_df = df
# Remove register withou CustomerID
#cs_df = cs_df[~(cs_df.CustomerID.isnull())]
cs_df.filter(cs_df.CustomerID.isNotNull())

# Remove negative or return transactions
cs_df = cs_df[~(cs_df.Quantity<0)]
cs_df = cs_df[cs_df.UnitPrice>0]


In [4]:
#refrence_date = cs_df.InvoiceDate.max() + datetime.timedelta(days = 1)
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql.types import StringType, IntegerType, StructType, StructField
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
from pyspark.sql.functions import col
max_date = cs_df.agg({'InvoiceDate': 'max'}).collect()[0]
max_date = max_date["max(InvoiceDate)"]
print('Max Date:', max_date)
cs_dfs = cs_df.withColumn("days_since_last_purchase", datediff(lit(max_date),cs_df.InvoiceDate))#.show(truncate=False)

#cs_dfs = cs_df.withColumn('days_since_last_purchase', F.datediff(F.to_date(F.date_add(F.to_date(lit(max_date)),1)), F.to_date(cs_df.InvoiceDate)))
#cs_dfs.filter(cs_dfs.days_since_last_purchase.isNotNull())

#cs_dfs.show()
customer_history_df =  cs_dfs[['CustomerID', 'days_since_last_purchase']].groupby("CustomerID").count()
customer_history_df.orderBy(customer_history_df["count"].desc())
customer_history_df = customer_history_df.select(col("CustomerID").alias("CustomerID"),col("count").alias("recency"))
customer_history_df.show()
#customer_history_df.describe().transpose()

Max Date: 9/9/2011 9:52
+----------+-------+
CustomerID|recency|
+----------+-------+
 16250| 18|
 15574| 139|
 15555| 534|
 15271| 211|
 17714| 10|
 17686| 259|
 13865| 29|
 14157| 38|
 13610| 183|
 13772| 135|
 13282| 35|
 12394| 23|
 16320| 38|
 13192| 47|
 14887| 5|
 17506| 11|
 17427| 2|
 18130| 46|
 16504| 41|
 15269| 2|
+----------+-------+
only showing top 20 rows

In [5]:
customer_freq = (cs_df[['CustomerID', 'InvoiceNo']].groupby(["CustomerID", 'InvoiceNo']).count().groupby(["CustomerID"]).count())
customer_history_df = customer_history_df.join(customer_freq, on=['CustomerID'], how='left_outer')
customer_history_df = customer_history_df.select(col("CustomerID").alias("CustomerID"),col("recency").alias("recency"),col("count").alias("frequency"))
customer_history_df.show()

+----------+-------+---------+
CustomerID|recency|frequency|
+----------+-------+---------+
 16250| 18| 2|
 15574| 139| 4|
 15555| 534| 16|
 15271| 211| 15|
 17714| 10| 1|
 17686| 259| 6|
 13865| 29| 4|
 14157| 38| 2|
 13610| 183| 7|
 13772| 135| 3|
 13282| 35| 3|
 12394| 23| 2|
 16320| 38| 2|
 13192| 47| 2|
 14887| 5| 1|
 17506| 11| 1|
 17427| 2| 1|
 18130| 46| 3|
 16504| 41| 1|
 15269| 2| 1|
+----------+-------+---------+
only showing top 20 rows

In [6]:
from pyspark.sql.functions import col
cs_df_amount = cs_df.withColumn("amount",col("Quantity") * col("UnitPrice"))#.groupby("CustomerID").sum()\
#(cs_df[['CustomerID', 'InvoiceNo']].groupby(["CustomerID", 'InvoiceNo']).count().groupby(["CustomerID"]).count())
customer_monetary_val = (cs_df_amount[['CustomerID', 'amount']].groupby(["CustomerID"]).sum())
customer_history_df = customer_history_df.join(customer_monetary_val, on=['CustomerID'], how='left_outer')
customer_history_df = customer_history_df.select(col("CustomerID").alias("CustomerID"),col("recency").alias("recency"),col("frequency").alias("frequency"),col("sum(amount)").alias("tot_amount"))
customer_history_df.show()

+----------+-------+---------+------------------+
CustomerID|recency|frequency| tot_amount|
+----------+-------+---------+------------------+
 16250| 18| 2| 319.5|
 15574| 139| 4| 642.6700000000001|
 15555| 534| 16|3541.1100000000006|
 15271| 211| 15|2237.1299999999997|
 17714| 10| 1| 153.0|
 17686| 259| 6| 5452.28|
 13865| 29| 4|494.96000000000004|
 14157| 38| 2| 363.9200000000001|
 13610| 183| 7|1068.0599999999997|
 13772| 135| 3|1011.7400000000002|
 13282| 35| 3|1111.6799999999998|
 12394| 23| 2| 1034.72|
 16320| 38| 2| 840.1|
 13192| 47| 2| 775.3200000000002|
 14887| 5| 1| 1790.0|
 17506| 11| 1| 270.7|
 17427| 2| 1| 100.8|
 18130| 46| 3| 799.8499999999999|
 16504| 41| 1|315.44999999999993|
 15269| 2| 1| 408.8|
+----------+-------+---------+------------------+
only showing top 20 rows

In [7]:
from pyspark.sql.functions import col
customer_history_df = customer_history_df.withColumn("recency_log", F.log(col("recency")))
customer_history_df = customer_history_df.withColumn("freq_log", F.log(col("frequency")))
customer_history_df = customer_history_df.withColumn("amt_log", F.log(col("tot_amount")))
feature_vector = ['amt_log', 'recency_log','freq_log']
X_subset = customer_history_df[feature_vector] #.as_matrix()
scaler = preprocessing.StandardScaler().fit(X_subset)
X_scaled = scaler.transform(X_subset)
pd.DataFrame(X_scaled, columns=X_subset.columns).describe().T

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-2255571884068052> in <module> 
 5 feature_vector = [ 'amt_log' , 'recency_log' , 'freq_log' ] 
 6 X_subset = customer_history_df [ feature_vector ] #.as_matrix() 
 ----> 7 scaler = preprocessing . StandardScaler ( ) . fit ( X_subset ) 
 8 X_scaled = scaler . transform ( X_subset ) 
 9 pd . DataFrame ( X_scaled , columns = X_subset . columns ) . describe ( ) . T

 /databricks/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py in fit (self, X, y) 

 /databricks/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py in partial_fit (self, X, y) 

 /databricks/python/lib/python3.7/site-packages/sklearn/utils/validation.py in check_array (array, accept_sparse, accept_large_sparse, dtype, order, copy, force_all_finite, ensure_2d, allow_nd, ensure_min_samples, ensure_min_features, warn_on_dtype, estimator) 
 525 UInt8Dtype , UInt16Dtype , 
 526 UInt32Dtype, UInt64Dtype)
 --> 527 if isinstance(dtype_iter, (Int8Dtype, Int16Dtype,
 528 Int32Dtype , Int64Dtype , 
 529 UInt8Dtype , UInt16Dtype , 

 /databricks/python/lib/python3.7/site-packages/numpy/core/numeric.py in asarray (a, dtype, order) 
 536 
 537 """
 --> 538 return array ( a , dtype , copy = False , order = order ) 
 539 
 540 

 ValueError : setting an array element with a sequence.